In [ ]:
import gym, ray
import numpy as np
import copy
import requests
from ray.rllib.agents.ppo import PPOTrainer

Note:

- This is a two-player game. So, maybe we could fix one player to always do the same thing, and then learn against that?
- Or, we could learn two agents?
- Or ???

In [ ]:
class IteratedPDEnv(gym.Env):
    
    done  = False
    
    def __init__ (self, env_config):
        # Note: Our action space is for ONE player; namely, exactly two
        # choices:
        #  - Defect
        #  - Cooperate
        self.action_space      = gym.spaces.Discrete(2)
        
        # We make no observations.
        self.observation_space = gym.spaces.Tuple((gym.spaces.Discrete(2), gym.spaces.Discrete(2)))

        """
        self.observation_space = gym.spaces.Box(
                                      low=0
                                    , high=np.iinfo(np.int32).max
                                    , shape=(1,)
                                    , dtype=np.int32
                                    )
        """
        
        # self.seed(1)
        self.reset()
    
    
    def reset (self):
        self.done = False
        return (0, 0)
    
    def action_to_int(self, action):
        if action == "Cooperate":
            return 0
        else:
            return 1
    
    def step (self, action):
        # action is either 0 or 1.
        
        if action == 0:
            player1Action = "Cooperate"
        else:
            player1Action = "Defect"
        
        player2Action = "Cooperate" # Goody-two-shoes
        
        assert action in [0, 1], "Unknown action!"
        
        # We are done once "step" is called; a round of the game is a single episode.
        self.done = True
        
        data = { "player1Action": player1Action
               , "player2Action": player2Action
               }
        
        # Do a post to the server; get the payoffs.
        response = requests.post("http://localhost:3000/play", json=data).json()
        
        reward = response["player1Payoff"]
        
        obs = (self.action_to_int (player1Action), self.action_to_int (player2Action))
        
        return [ obs, reward, self.done, response ]

In [ ]:
env = IteratedPDEnv(env_config = {})

In [ ]:
env.step(action=1)

### Let's try training it!

In [ ]:
# ray.init(local_mode=True)

In [ ]:
trainer = PPOTrainer(env=IteratedPDEnv, config={
    "framework": "tf2",
    "num_workers": 1,
    "env_config": {},
    "create_env_on_driver": True
})

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()